In [1]:
from flask import Flask,render_template,url_for,request
import pandas as pd 
import pickle

In [3]:
from ipywidgets import widgets, Layout, Label
from IPython.display import display, clear_output
import numpy as np
from nltk.tokenize import sent_tokenize
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import re
import string
import unicodedata
import nltk

from termcolor import colored

In [4]:
def remove_url(str):
    str = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', str)
    return str
    
def remove_digit(str):
    str = re.sub(r'[^a-z ]*([.0-9])*\d', ' ', str)
    return str 

def remove_non_ascii(str):
    str = unicodedata.normalize('NFKD', str).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return str

def remove_twitter_char(str):  
    # mention
    str = re.sub(r'(?:@[\w_]+)', ' ', str)
    # hashtag
    str = re.sub(r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", " ", str)
    # RT/cc
    str = re.sub('RT', ' ', str)

    return str

def remove_punctuation(str):
    str = re.sub(r'[^\s\w]', ' ', str)
    return str

def remove_multi_space(str):
    str = re.sub('[\s]+', ' ', str)
    return str

def casefolding(str):  
    str = str.lower()   
    return ' '.join(str.split())

def remove_repeated_character(str):
    str = re.sub(r'(.)\1{2,}', r'\1', str)
    return str

def normalize_slang_word(str):
    alay_dict = pd.read_csv('data/slang_word_list.csv', encoding='latin-1', header=None)
    alay_dict = alay_dict.rename(columns={0: 'original', 1: 'replacement'})
    alay_dict_map = dict(zip(alay_dict['original'], alay_dict['replacement']))
    return ' '.join([alay_dict_map[word] if word in alay_dict_map else word for word in str.split(' ')])

def remove_laugh(str):
    str = re.sub(r"\b(?:(h|a|e)*(?:(ha|he|hue))+h?|(?:l+o+)+l+)|(?:(w|k)*(?:wk)+(w?|k?))\b", ' ', str)
    
    return str

def remove_unused_char(str):
    tmp = ""    
    if len(str) < 3:
        tmp
    else:
        tmp = str
        
    return tmp

In [5]:
def preprocessing(str):
    str = remove_url(str)
    str = remove_twitter_char(str)
    str = remove_digit(str)
    str = remove_non_ascii(str)
    str = remove_punctuation(str)
    str = remove_laugh(str)
    str = remove_multi_space(str) 
    str = remove_repeated_character(str)    
    str = casefolding(str)
    str = normalize_slang_word(str)
    str = remove_unused_char(str)
    
    return str

In [6]:
train = pd.read_excel("dataset_clean/train_preprocess.xlsx")
train.replace('', np.nan, inplace=True)
train.dropna(inplace=True) # memastikan tidak ada nilai yang kosong

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train['sentence'].values)
maxlen = max([len(i.split()) for i in train['sentence'].values])

In [10]:
app = Flask(__name__, template_folder = r"C:\Users\PREDATOR\Downloads\uji Coba")

model = load_model('model.h5') 

@app.route('/')
def feedback():
    return render_template('feedback_pred.html')

@app.route('/result',methods=['POST','GET'])
def result():

    if request.method == 'POST':
        kalimat = request.form['Data']
        print(kalimat)
        kalimat = preprocessing(kalimat)
        kalimat = sent_tokenize(kalimat)
        kalimat = tokenizer.texts_to_sequences(kalimat)
        kalimat = pad_sequences(kalimat, maxlen=maxlen)
        pred = model.predict(kalimat)
        print(pred)
        if(np.argmax(pred) == 0):
            return render_template('feedback_result.html',result = "positive")
        elif (np.argmax(pred) == 1):
            return render_template('feedback_result.html',result = "negative")



if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [25/May/2022 07:33:14] "GET / HTTP/1.1" 200 -


nama saya raihan


127.0.0.1 - - [25/May/2022 07:33:22] "POST /result HTTP/1.1" 200 -


[[0.95557064 0.04381523]]


...

muka lu kayak kontol
[[9.1072041e-05 9.9991035e-01]]


127.0.0.1 - - [25/May/2022 07:33:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 07:34:05] "POST /result HTTP/1.1" 200 -


anjing kontol bangsat
[[9.991025e-01 8.905232e-04]]


127.0.0.1 - - [25/May/2022 07:35:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2022 07:35:44] "POST /result HTTP/1.1" 200 -


tolol banget jadi orang
[[0.9984255  0.00155509]]


NameError: name 'pred' is not defined

In [12]:
print("feedback detected")
button = widgets.ToggleButton(description="Check")
text = widgets.Text(layout=Layout(width='80%'))
output = widgets.Output()

def on_button_clicked(b):
    output.clear_output()      
    with output:
        kalimat = text.value
        kalimat = preprocessing(kalimat)
        kalimat = sent_tokenize(kalimat)
        kalimat = tokenizer.texts_to_sequences(kalimat)
        kalimat = pad_sequences(kalimat, maxlen=maxlen)
        pred = model.predict(kalimat)  
        print("Status Konten: ")
        if(np.argmax(pred) == 0):
            non_adult = colored('Positive', color='grey', on_color='on_green', attrs=['bold'])
            print(non_adult)
        elif (np.argmax(pred) == 1):
            adult = colored('Negative', color='grey', on_color='on_red', attrs=['bold'])
            print(adult)

display(text)
display(button, output)
button.observe(on_button_clicked)

feedback detected


Text(value='', layout=Layout(width='80%'))

ToggleButton(value=False, description='Check')

Output()

In [13]:
df = pd.DataFrame(pred, columns = ['ouput'])

cols = prediction[0].keys()
df = pd.DataFrame([[getattr(i,j) for j in cols] for i in prediction], columns = cols)

NameError: name 'pred' is not defined